In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.drawing.image import Image
import win32com.client as win32
import os

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns

In [2]:
# manual_sending = False # True/False Заполните это поле, если хотите отправить отчет даже после критичных уведомлений

# Print_qualuty_check = True # Вынести QC в отдельный excel-файл? True/False
# Display_QC_mail = True # Показать письмо QC для отправки? True/False
# Send_QC_mail = True # Создать и отправить письмо для QC? True/False

Print_to_excel = True # Создать excel-файл с расчетами? True/False
# Display_mail = True # Показать письмо для отправки? True/False
# Send_mail = True # Создать и отправить письмо с расчетами и графиком? True/False

# mail_to = 'TarakanovMIu@aimmngt.com' # Получатель письма

In [40]:
query = f"""
SELECT *
FROM "RISKACCESS"."bankAccountsBalanceDaily"
WHERE "reportDate" = (SELECT MAX("reportDate") FROM "RISKACCESS"."bankAccountsBalanceDaily")
"""
bankAccountsBalanceDaily_data = export_from_RISKCUSTOM(query) # export data
BABD_data_work = bankAccountsBalanceDaily_data # BABD_data_work,
BABD_data_work['balanceUsd_mln'] = BABD_data_work.balanceUsd/10**6 # balanceUsd_mln
# balanceUsd_activmoney_market
accountStatus_list = ['active', 'mmarket']
BABD_data_work['balanceUsd_activmoney_market'] = 0
BABD_data_work.loc[BABD_data_work.accountStatus.isin(accountStatus_list), 'balanceUsd_activmoney_market'] = BABD_data_work.loc[BABD_data_work.accountStatus.isin(accountStatus_list), 'balanceUsd_mln']

query = f"""
SELECT t.*
FROM "RISKACCESS"."xxmrBankLimits" t
WHERE "limitType" in ('Transit', 'Deposit')
AND "activeFrom" = (
  SELECT MAX("activeFrom")
  FROM "RISKACCESS"."xxmrBankLimits" t2
  WHERE t2."bankId" = t."bankId"
  GROUP BY t2."bankId"
)"""
Limit_export_data = export_from_RISKCUSTOM(query) # export data
LE_data_work = Limit_export_data # LE_data_work
# pivot by bankid and mln of limit
LE_data_group = pd.pivot_table(data=LE_data_work, index=['holding', 'bankId'], values='limit', aggfunc='sum').reset_index()
LE_data_group.limit = LE_data_group.limit/10**6
# merge BABD_data_work and LE_data_group
LE_data_group['H_B_concat'] = LE_data_group.holding + '_' + LE_data_group.bankId
BABD_data_work = BABD_data_work.reset_index(drop=True)
BABD_data_work['H_B_concat'] = BABD_data_work.holding + '_' + BABD_data_work.bankId
BABD_data_work['Limit'] = BABD_data_work.merge(LE_data_group, how='left', left_on='H_B_concat', right_on='H_B_concat', validate='many_to_one').iloc[:,-1]
# Usage and Usage_activmoney_market
BABD_data_work['Usage_activmoney_market'] = 0
BABD_data_work['Usage'] = 0
BABD_data_limit_not_na = BABD_data_work[~BABD_data_work.Limit.isna()]
BABD_data_limit_not_na['Usage_activmoney_market'] = (BABD_data_limit_not_na.balanceUsd_activmoney_market/BABD_data_limit_not_na.Limit)*100
BABD_data_limit_not_na['Usage'] = (BABD_data_limit_not_na.balanceUsd_mln/BABD_data_limit_not_na.Limit)*100
BABD_data_work[~BABD_data_work.Limit.isna()] = BABD_data_limit_not_na
BABD_data_work.loc[(BABD_data_work.Usage == np.inf) | (BABD_data_work.Usage.isna()), 'Usage'] = 0
BABD_data_work.loc[(BABD_data_work.Usage_activmoney_market == np.inf) | (BABD_data_work.Usage_activmoney_market.isna()), 'Usage_activmoney_market'] = 0
BABD_data_work['Segment'] = merge_SalesUnits(df=BABD_data_work, merge_col='businessSegmentDetailed', col='buCode') # merge Segment

BABD_data_work = BABD_data_work.rename(columns={'balanceUsd_activmoney_market':'Active', 'balanceUsd_mln':'Total', 'Usage_activmoney_market':'%_active', 'Usage':'%'})
# Creating Excel Writer Object from Pandas 
report_date = str(BABD_data_work.reportDate.max())[:10]
Output_file = report_date + '_limits_report.xlsx'
if Print_to_excel == True:
    writer = pd.ExcelWriter(Output_file, engine='openpyxl')  
    workbook=writer.book
# by holding
holding_list = BABD_data_work.holding.unique().tolist()
for holding in holding_list:
    holding_data = BABD_data_work[BABD_data_work.holding == holding].reset_index(drop=True) # holding data
    # table 1
    tabel_bankName = pd.pivot_table(data=holding_data, 
                    index='bankName', 
                    values=['Limit', 'Active', 'Total', '%_active', '%'], 
                    aggfunc={'Limit':'mean', 'Active':'sum', 'Total':'sum', '%_active':'sum', '%':'sum'},
                    fill_value=0)\
                    .reset_index()\
                    .sort_values('Active', ascending=False)
    # table 2
    table_bankCountryCode = pd.pivot_table(data=holding_data, 
                    index='bankCountryCode', 
                    values=['Active', 'Total'], 
                    aggfunc={'Active':'sum', 'Total':'sum'})\
                    .reset_index()\
                    .sort_values('Active', ascending=False)
    # table 3
    table_Segment = pd.pivot_table(data=holding_data, 
                    index='Segment', 
                    values=['Active', 'Total'], 
                    aggfunc={'Active':'sum', 'Total':'sum'})\
                    .reset_index()\
                    .sort_values('Active', ascending=False)
    # to excel
    if Print_to_excel == True:
            tabel_bankName.to_excel(writer, sheet_name=holding, index=False) # to excel
            table_bankCountryCode.to_excel(writer, sheet_name=holding, startcol=8, index=False) # to excel
            table_Segment.to_excel(writer, sheet_name=holding, startcol=13, index=False) 
if Print_to_excel == True:
    writer.close()

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11716\578256563.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.00450528 0.00424549 0.05693758 ... 0.         0.         0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  BABD_data_work.loc[BABD_data_work.accountStatus.isin(accountStatus_list), 'balanceUsd_activmoney_market'] = BABD_data_work.loc[BABD_data_work.accountStatus.isin(accountStatus_list), 'balanceUsd_mln']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connecta

In [4]:
manual_map = BABD_data_work.loc[BABD_data_work['Segment'] == 'External', ['holding', 'buCode']]
manual_map

,holding,buCode
1482,EUROCHEM,AEMNG
1483,EUROCHEM,AEMNG


In [26]:
BABD_data_work[BABD_data_work.holding == 'SUEK']

,##batch_id,##deleted,##inserted,##origin_name,reportDate,holding,buCode,buCountryGroup,buCountryCode,buName,...,balance,balanceUsd,statementDate,balanceUsd_mln,balanceUsd_activmoney_market,H_B_concat,Limit,Usage_activmoney_market,Usage,Segment
2195,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,V800,(0) RU,RU,"АО ""Кызылская ТЭЦ""",...,0.00,0.00000,2023-05-01,0.000000,0.000000,SUEK_049304735,NaN,0.0,0.0,SGC
2196,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,4100,(0) RU,RU,"ООО ""Восточно-Бейский раз",...,0.00,0.00000,2022-05-30,0.000000,0.000000,SUEK_ABSLRU,NaN,0.0,0.0,SUEK RU
2197,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,4100,(0) RU,RU,"ООО ""Восточно-Бейский раз",...,0.00,0.00000,2022-05-30,0.000000,0.000000,SUEK_ABSLRU,NaN,0.0,0.0,SUEK RU
2198,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,4100,(0) RU,RU,"ООО ""Восточно-Бейский раз",...,0.00,0.00000,2022-05-30,0.000000,0.000000,SUEK_ABSLRU,NaN,0.0,0.0,SUEK RU
2199,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,4100,(0) RU,RU,"ООО ""Восточно-Бейский раз",...,0.00,0.00000,2022-05-30,0.000000,0.000000,SUEK_ABSLRU,NaN,0.0,0.0,SUEK RU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,1100,(0) RU,RU,"АО ""СУЭК""",...,0.00,0.00000,2022-04-21,0.000000,0.000000,SUEK_ZENIRU,NaN,0.0,0.0,SUEK RU
3673,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,1100,(0) RU,RU,"АО ""СУЭК""",...,136378.39,1485.94168,2024-04-27,0.001486,0.001486,SUEK_ZENIRU,NaN,0.0,0.0,SUEK RU
3674,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,1100,(0) RU,RU,"АО ""СУЭК""",...,0.00,0.00000,2022-06-20,0.000000,0.000000,SUEK_ZENIRU,NaN,0.0,0.0,SUEK RU
3675,36115663@WHPROD,N,2024-05-03 12:05:23,WHFLEX,2024-04-27,SUEK,1100,(0) RU,RU,"АО ""СУЭК""",...,0.00,0.00000,2022-06-24,0.000000,0.000000,SUEK_ZENIRU,NaN,0.0,0.0,SUEK RU


In [39]:

# query = f"""
# SELECT *
# FROM "RISKACCESS"."xxmrBankLimits"
# ORDER BY "activeFrom" DESC
# FETCH FIRST 10 ROWS ONLY
# """
query = """SELECT t.*
FROM "RISKACCESS"."xxmrBankLimits" t
WHERE "limitType" in ('Transit', 'Deposit')
AND "activeFrom" = (
  SELECT MAX("activeFrom")
  FROM "RISKACCESS"."xxmrBankLimits" t2
  WHERE t2."bankId" = t."bankId"
  GROUP BY t2."bankId"
)"""


Limit_export_data = export_from_RISKCUSTOM(query) # export data
Limit_export_data
BABD_data_work.merge(LE_data_group, how='inner', left_on='H_B_concat', right_on='H_B_concat', validate='many_to_one')

# LE_data_work = Limit_export_data # LE_data_work
# # limitType filter
# limitType_list = ['Transit', 'Deposit']
# LE_data_work = LE_data_work[LE_data_work.limitType.isin(limitType_list)]
# # pivot by bankid and mln of limit
# LE_data_group = pd.pivot_table(data=LE_data_work, index=['holding', 'bankId'], values='limit', aggfunc='sum').reset_index()
# LE_data_group.limit = LE_data_group.limit/10**6

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,activeFrom,bankId,limitType,limit,createdAt,createdBy,updatedAt,updatedBy,holding,nextRevisionDate
0,2000-01-01,ALSABH,Transit,10000000.0,2022-10-11 13:15:41,None,2022-10-11 13:15:41,None,SUEK,NaT
1,2000-01-01,ALSABH,Transit,10000000.0,2022-10-11 13:15:41,None,2022-10-11 13:15:41,None,EUROCHEM,NaT
2,2000-01-01,BMCEMA,Transit,50000000.0,2022-10-11 13:15:41,None,2022-10-11 13:15:41,None,SUEK,NaT
3,2000-01-01,BMCEMA,Transit,50000000.0,2022-10-11 13:15:41,None,2022-10-11 13:15:41,None,EUROCHEM,NaT
4,2023-05-16,ICBKCN,Deposit,200000000.0,2022-10-11 13:15:41,TARAKANOVMIU@SUEK.RU,2023-05-16 15:23:50,LAGNOAO@SUEK.RU,SUEK,NaT
...,...,...,...,...,...,...,...,...,...,...
298,2024-02-13,COBACH,Transit,0.0,2024-02-13 11:57:23,TARAKANOVMIU@SUEK.RU,2024-02-13 11:57:23,TARAKANOVMIU@SUEK.RU,EUROCHEM,NaT
299,2024-03-15,BOSHCN,Deposit,200000000.0,2022-12-02 12:29:13,TARAKANOVMIU@SUEK.RU,2024-03-15 10:40:59,TIMOKHINEV@SUEK.RU,EUROCHEM,NaT
300,2024-03-15,KSNVKZ,Transit,25000000.0,2022-10-11 13:15:41,TARAKANOVMIU@SUEK.RU,2024-03-15 13:31:10,TIMOKHINEV@SUEK.RU,EUROCHEM,NaT
301,2023-04-26,AGRICH,Transit,NaN,2022-10-11 13:15:41,TARAKANOVMIU@SUEK.RU,2023-04-26 13:53:17,TARAKANOVMIU@SUEK.RU,SUEK,NaT
